In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

## load the dataset

In [2]:
data = pd.read_csv('data/Churn_Modelling.csv')
data.head()

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         79084.10       0

In [3]:
## preprocess the data
# drop irrelevant columns
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
data

CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0             619    France  Female   42       2       0.00              1   
1             608     Spain  Female   41       1   83807.86              1   
2             502    France  Female   42       8  159660.80              3   
3             699    France  Female   39       1       0.00              2   
4             850     Spain  Female   43       2  125510.82              1   
...           ...       ...     ...  ...     ...        ...            ...   
9995          771    France    Male   39       5       0.00              2   
9996          516    France    Male   35      10   57369.61              1   
9997          709    France  Female   36       7       0.00              1   
9998          772   Germany    Male   42       3   75075.31              2   
9999          792    France  Female   28       4  130142.79              1   

      HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0             1               1        101348.88       1  
1             0               1        112542.58       0  
2             1               0        113931.57       1  
3             0               0         93826.63       0  
4             1               1         79084.10       0  
...         ...             ...              ...     ...  
9995          1               0         96270.64       0  
9996          1               1        101699.77       0  
9997          0               1         42085.58       1  
9998          1               0         92888.52       1  
9999          1               0         38190.78       0  

[10000 rows x 11 columns]

In [5]:
# encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0             619    France       0   42       2       0.00              1   
1             608     Spain       0   41       1   83807.86              1   
2             502    France       0   42       8  159660.80              3   
3             699    France       0   39       1       0.00              2   
4             850     Spain       0   43       2  125510.82              1   
...           ...       ...     ...  ...     ...        ...            ...   
9995          771    France       1   39       5       0.00              2   
9996          516    France       1   35      10   57369.61              1   
9997          709    France       0   36       7       0.00              1   
9998          772   Germany       1   42       3   75075.31              2   
9999          792    France       0   28       4  130142.79              1   

      HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0             1               1        101348.88       1  
1             0               1        112542.58       0  
2             1               0        113931.57       1  
3             0               0         93826.63       0  
4             1               1         79084.10       0  
...         ...             ...              ...     ...  
9995          1               0         96270.64       0  
9996          1               1        101699.77       0  
9997          0               1         42085.58       1  
9998          1               0         92888.52       1  
9999          1               0         38190.78       0  

[10000 rows x 11 columns]

In [10]:
# One hot encode the 'Geography' column
oneHotEncoder_geo = OneHotEncoder()
geo_encoder = oneHotEncoder_geo.fit_transform(data[['Geography']])
geo_encoder

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [13]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [ ]:
oneHotEncoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [14]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=oneHotEncoder_geo.get_feature_names_out(['Geography'])) 

In [15]:
geo_encoded_df

Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]

In [16]:
# combine the encoded geography with the original data
data = pd.concat([data, geo_encoded_df], axis=1)
# drop the original 'Geography' column
data.drop(['Geography'], axis=1, inplace=True)

data

CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0             619       0   42       2       0.00              1          1   
1             608       0   41       1   83807.86              1          0   
2             502       0   42       8  159660.80              3          1   
3             699       0   39       1       0.00              2          0   
4             850       0   43       2  125510.82              1          1   
...           ...     ...  ...     ...        ...            ...        ...   
9995          771       1   39       5       0.00              2          1   
9996          516       1   35      10   57369.61              1          1   
9997          709       0   36       7       0.00              1          0   
9998          772       1   42       3   75075.31              2          1   
9999          792       0   28       4  130142.79              1          1   

      IsActiveMember  EstimatedSalary  Exited  Geography_France  \
0                  1        101348.88       1               1.0   
1                  1        112542.58       0               0.0   
2                  0        113931.57       1               1.0   
3                  0         93826.63       0               1.0   
4                  1         79084.10       0               0.0   
...              ...              ...     ...               ...   
9995               0         96270.64       0               1.0   
9996               1        101699.77       0               1.0   
9997               1         42085.58       1               1.0   
9998               0         92888.52       1               0.0   
9999               0         38190.78       0               1.0   

      Geography_Germany  Geography_Spain  
0                   0.0              0.0  
1                   0.0              1.0  
2                   0.0              0.0  
3                   0.0              0.0  
4                   0.0              1.0  
...                 ...              ...  
9995                0.0              0.0  
9996                0.0              0.0  
9997                0.0              0.0  
9998                1.0              0.0  
9999                0.0              0.0  

[10000 rows x 13 columns]

In [17]:
# save encoders and scaler
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('onehot_encoder_geo.pkl', 'wb') as f:
    pickle.dump(oneHotEncoder_geo, f)

In [18]:
data.head()

CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0          619       0   42       2       0.00              1          1   
1          608       0   41       1   83807.86              1          0   
2          502       0   42       8  159660.80              3          1   
3          699       0   39       1       0.00              2          0   
4          850       0   43       2  125510.82              1          1   

   IsActiveMember  EstimatedSalary  Exited  Geography_France  \
0               1        101348.88       1               1.0   
1               1        112542.58       0               0.0   
2               0        113931.57       1               1.0   
3               0         93826.63       0               1.0   
4               1         79084.10       0               0.0   

   Geography_Germany  Geography_Spain  
0                0.0              0.0  
1                0.0              1.0  
2                0.0              0.0  
3                0.0              0.0  
4                0.0              1.0

In [ ]:
# divide the data into independent and dependent variables
# Exited is the dependent variable
X = data.drop(['Exited'], axis=1)
y = data['Exited']

# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale the features
# look up StandardScaler in sklearn
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
# saving scaler as pickle file
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [22]:
data

CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0             619       0   42       2       0.00              1          1   
1             608       0   41       1   83807.86              1          0   
2             502       0   42       8  159660.80              3          1   
3             699       0   39       1       0.00              2          0   
4             850       0   43       2  125510.82              1          1   
...           ...     ...  ...     ...        ...            ...        ...   
9995          771       1   39       5       0.00              2          1   
9996          516       1   35      10   57369.61              1          1   
9997          709       0   36       7       0.00              1          0   
9998          772       1   42       3   75075.31              2          1   
9999          792       0   28       4  130142.79              1          1   

      IsActiveMember  EstimatedSalary  Exited  Geography_France  \
0                  1        101348.88       1               1.0   
1                  1        112542.58       0               0.0   
2                  0        113931.57       1               1.0   
3                  0         93826.63       0               1.0   
4                  1         79084.10       0               0.0   
...              ...              ...     ...               ...   
9995               0         96270.64       0               1.0   
9996               1        101699.77       0               1.0   
9997               1         42085.58       1               1.0   
9998               0         92888.52       1               0.0   
9999               0         38190.78       0               1.0   

      Geography_Germany  Geography_Spain  
0                   0.0              0.0  
1                   0.0              1.0  
2                   0.0              0.0  
3                   0.0              0.0  
4                   0.0              1.0  
...                 ...              ...  
9995                0.0              0.0  
9996                0.0              0.0  
9997                0.0              0.0  
9998                1.0              0.0  
9999                0.0              0.0  

[10000 rows x 13 columns]

## ANN implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from datetime import datetime

In [30]:
X_train.shape[1] # number of input features

12

In [33]:
# Build the ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # single dimensional input, HL1 connected with input layer
    Dense(32, activation='relu'), # HL2
    Dense(1, activation='sigmoid') # output layer, binary classification
])

/home/bionic/code/nlp-tutorial/env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
E0000 00:00:1754846011.047571    6702 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1754846011.049725    6702 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [34]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x7fa948d21d00>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [37]:
# forward and backward propagation
# compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
# set up tensorboard logging
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [42]:
# setup early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) # wait till 5 epochs without improvement
# consider the best weights in between the epochs

In [43]:
# training the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test), # validation data for early stopping
                    epochs=100, # number of epochs
                    callbacks=[tensorflow_callback, early_stopping_callback]
                ) # callbacks for tensorboard and early stopping

Epoch 1/100


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8649 - loss: 0.3263 - val_accuracy: 0.8630 - val_loss: 0.3409
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8650 - loss: 0.3244 - val_accuracy: 0.8625 - val_loss: 0.3399
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8676 - loss: 0.3225 - val_accuracy: 0.8600 - val_loss: 0.3451
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8654 - loss: 0.3205 - val_accuracy: 0.8680 - val_loss: 0.3398
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8670 - loss: 0.3187 - val_accuracy: 0.8600 - val_loss: 0.3429
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8669 - loss: 0.3182 - val_accuracy: 0.8605 - val_loss: 0.3394
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8665 - loss: 0.3165 - val_accuracy: 0.8605 - val_loss: 0.3429
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8658 - loss: 0.3159 - val_accuracy: 0.8610

In [44]:
model.save('model.h5')

In [46]:
# load tensorboard extension
%load_ext tensorboard

In [47]:
%tensorboard --logdir logs/fit --host localhost --port 6006

In [ ]:
# load the pickle files
import pickle

